a, b と改行で並べるだけで出力できるように設定します

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

`data/input/train.csv` のデータを元に主要な統計量を計算します。

In [ ]:
from numpy import test
import pandas as pd
import sys
from pathlib import Path
# codeフォルダへのパスを追加
sys.path.append(str(Path("..").resolve()))
from python.path_utils import input_path

# パス設定
INPUT_PATH = input_path("train.csv")
TEST_PATH = input_path("test.csv")

# データ読み込み
df = pd.read_csv(INPUT_PATH)
test_df = pd.read_csv(TEST_PATH)

dfの内容を詳しく見ます。

In [ ]:
# 構造の把握
df
df.shape          # (行数, 列数)
df.dtypes         # 各列の型
df.info()         # 型 + 非null数 + メモリ使用量を一覧表示
df.describe()     # 数値列の統計量（count, mean, std, min, max, 四分位）

# データの中身を見る
df.head()         # 先頭5行（引数で行数指定可）
df.tail(10)       # 末尾10行
df.sample(5)      # ランダムに5行（偏りなく雰囲気を掴める）

# 欠損・ユニーク値
df.isnull().sum()          # 列ごとの欠損数
df["Age"].value_counts()   # カテゴリ列の分布
df.nunique()               # 列ごとのユニーク値数

In [ ]:
df.isnull().sum() 

Age・cabin・embarked列には欠損値があることがわかります。
それらのカラムのデータ型を確認しましょう。

In [ ]:
df.dtypes[df.isnull().sum() > 0]

Ageは数値列、cabinとembarkedは文字列列であることがわかります。
Age列は数値が入っているため、平均や中央値などの統計量を計算できます。

In [ ]:
df.nunique()[df.isnull().sum() > 0]

文字列型のCabinとEmbarkedですが、Embarkedはユニークな値が3つしかないため、カテゴリ列として扱うことができます。
ひとまず、最も多い値で欠損値を埋めることにしましょう。

In [ ]:
df["Embarked"].unique()

In [ ]:
# Embarkedの最頻値で欠損を補完しカテゴリ型に変換
embarked_mode = df["Embarked"].mode(dropna=True)[0]
# 欠損値を最頻値で埋めてからカテゴリ型に変換
# カテゴリ型に変換することで、モデルが文字列を数値として扱えるようになります。（ENUM型のようなもの）
df["Embarked"] = df["Embarked"].fillna(embarked_mode).astype("category")

# 確認
embarked_mode
df["Embarked"].isnull().sum()
df["Embarked"].value_counts()

In [ ]:
# Ageの平均値で欠損を補完
age_mean = df["Age"].mean()
df["Age"] = df["Age"].fillna(age_mean)

# 確認
age_mean
df["Age"].isnull().sum()
df["Age"].describe()

Cabin列はユニークな値も多く、欠損値も多いため、今回は削除してしまいましょう。

In [ ]:
# Cabin列を削除
df = df.drop(columns=["Cabin"] )

# 確認
"Cabin" in df.columns, df.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 元データからモデルをトレーニング
y = df["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(df[features])
model = RandomForestClassifier(n_estimators=10, max_depth=1, random_state=1)
model.fit(X, y)

# テストデータから予測
X_test = pd.get_dummies(test_df[features])
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)